In [123]:
import pandas as pd
import matplotlib.pyplot as plt
from utils.Points import PlayerPoints, PlayerServes
from utils.Tools import Victor,ServStats,PtWinner,Server,Shots, PtEnding, Rallys
import numpy as np
import os
import csv
import warnings

In [124]:
warnings.filterwarnings('ignore')

In [125]:
SymbolsDictionary = {"F" : "Forehand", 
                     "B" : "Backhand", 
                     "R" :"FH Slice", 
                     "S" : "BH Slice", 
                     "V" : "FH Volley", 
                     "Z" : "BH Volley"}

EndDict = {"*": ("Winner","Ace"),
           "@": "Unforced Error",
           "#" : "Forced Error"}

In [ ]:
def Builder(Player):
    player_name = Player.split(' ',1)[1]
    
    directory_name = player_name +'_data'

    try:
        os.mkdir(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
    except FileExistsError:
        print(f"Directory '{directory_name}' already exists.")
    except PermissionError:
        print(f"Permission denied: Unable to create '{directory_name}'.")
    except Exception as e:
        print(f"An error occurred: {e}")

    matches = pd.read_csv('BaseData/charting-m-matches.csv', encoding='unicode_escape',quoting=csv.QUOTE_NONE)
    matches = pd.read_csv('BaseData/charting-m-matches.csv', encoding='unicode_escape',quoting=csv.QUOTE_NONE)

    matches2012 = matches[matches['Date'].str[:4] == '2012']
    matches2012 = matches2012

    PlayerGames = matches2012[(matches2012['Player 1'] == Player) | (matches2012['Player 2'] == Player)].reset_index(drop=True)

    Points = pd.read_csv('ProjData/Points.csv')

    PlayerAux = PlayerGames[['match_id','Player 1', 'Player 2','Surface']]

    PointsPlayer = pd.merge(Points,PlayerAux, on='match_id')

    Victor(PointsPlayer,PlayerGames)

    PlayerGames['Winner'] = PlayerGames['Winner'].apply(lambda x: True if x == "["+Player+"]" else False)

    PointsPlayer['Victor'] = PtWinner(PointsPlayer,Player, PlayerGames)

    PointsPlayer['Server'] = Server(PointsPlayer,Player, PlayerGames)

    PointsPlayer[['Ace','Winner','Unforced Error','Forced Error','Double Fault']] = None

    PointsPlayer.loc[(PointsPlayer['2nd'].isna()), ['2nd']] = None

    PtEnding(PointsPlayer,EndDict)

    PlayerGames.to_csv(directory_name+'/'+ player_name  +'2012.csv',index=False)
    PointsPlayer.to_csv(directory_name+'/Points'+ player_name  +'.csv',index=False)

    Rally = pd.read_csv('BaseData/charting-m-stats-Rally.csv')
    ShotTypes = pd.read_csv('BaseData\charting-m-stats-ShotTypes.csv')
    Serves = pd.read_csv('BaseData/charting-m-stats-ServeBasics.csv')
    ShotDir = pd.read_csv('BaseData/charting-m-stats-ShotDirection.csv')

    PlayerGamesRally = Rally[Rally['match_id'].isin(PlayerGames['match_id'])]
    # PlayerGamesServes = Serves[Serves['match_id'].isin(PlayerGames['match_id'])]
    PlayerGamesShotTypes = ShotTypes[ShotTypes['match_id'].isin(PlayerGames['match_id'])]
    PlayerGamesShotDir = ShotDir[ShotDir['match_id'].isin(PlayerGames['match_id'])]

    rowsServe = ['Total','1-3','4-6','7-9','10']
    rowsIndx = ['match_id','1-3','4-6','7-9','10-']
    
    PlayerRallyServe = PlayerGamesRally[PlayerGamesRally['server'] == Player].drop(columns=['server','returner'])
    PlayerRallyServe = PlayerRallyServe[~PlayerRallyServe['row'].isin(rowsServe)]
    PlayerRallyServe['row'] = PlayerRallyServe['row'].str[:3]
    PlayerRallyServe = Rallys(PlayerRallyServe,rowsIndx)

    PlayerRallyServe.to_csv(directory_name +'/'+player_name +'ServesRally.csv',index=False)

    PlayerRallyReturn = PlayerGamesRally[PlayerGamesRally['returner'] == Player].drop(columns=['server','returner'])
    PlayerRallyReturn = PlayerRallyReturn[~PlayerRallyReturn['row'].isin(rowsServe)]
    PlayerRallyReturn['row'] = PlayerRallyReturn['row'].str[:3]
    PlayerRallyReturn = Rallys(PlayerRallyReturn,rowsIndx)
    PlayerRallyReturn.to_csv(directory_name +'/'+player_name +'Returns.csv',index=False)

    PlayerShotDirTotal = PlayerGamesShotDir[PlayerGamesShotDir['player'] == Player]
    PlayerShotDirTotal = PlayerShotDirTotal[PlayerShotDirTotal['row'] == 'Total']
    PlayerShotDirTotal = PlayerShotDirTotal.drop(columns=['player','row'])
    PlayerShotDirTotal.to_csv(directory_name +'/'+player_name +'ShotDir.csv',index=False)

    rowsST = ['match_id','Total', 'F', 'B', 'R', 'S', 'V','Z']
    PlayerGamesShotTypes = PlayerGamesShotTypes[PlayerGamesShotTypes['row'].isin(rowsST)].reset_index(drop=True)
    PlayerShotTypes = PlayerGamesShotTypes[PlayerGamesShotTypes['player'] == Player]

    PlayerShotStats = Shots(PlayerShotTypes,rows=rowsST)
    PlayerShotStats = PlayerShotStats.rename(columns=SymbolsDictionary)
    PlayerShotStats = PlayerShotStats.fillna(0)
    


    PlayerShotStats['Win'] = PlayerGames['Winner']
    
    PlayerShotStats.to_csv(directory_name +'/'+player_name +'ShotType.csv',index=False)
    # data = PlayerShotStats[[ 'Total', 'Forehand', 'Backhand', 'FH Slice', 'BH Slice',
    #     'FH Volley', 'BH Volley', 'Win']]

    PlayerServes = ServStats(Serves,Player,PlayerGames)

    PlayerServes.to_csv(directory_name +'/'+player_name +'Serve.csv',index=False)

    

In [127]:
Builder('Roger Federer')

Directory 'Federer_data' already exists.


In [128]:
    # directory_name = player +'_data'

    # try:
    #     os.mkdir(directory_name)
    #     print(f"Directory '{directory_name}' created successfully.")
    # except FileExistsError:
    #     print(f"Directory '{directory_name}' already exists.")
    # except PermissionError:
    #     print(f"Permission denied: Unable to create '{directory_name}'.")
    # except Exception as e:
    #     print(f"An error occurred: {e}")

In [129]:
# matches = pd.read_csv('BaseData/charting-m-matches.csv', encoding='unicode_escape',quoting=csv.QUOTE_NONE)
# Base = pd.read_csv('BaseData/atp_matches_2012.csv',encoding='unicode_escape',quoting=csv.QUOTE_NONE)

In [130]:
    # matches = pd.read_csv('BaseData/charting-m-matches.csv', encoding='unicode_escape',quoting=csv.QUOTE_NONE)

    # matches2012 = matches[matches['Date'].str[:4] == '2012']
    # matches2012 = matches2012

    # FedererGames = matches2012[(matches2012['Player 1'] == 'Roger Federer') | (matches2012['Player 2'] == 'Roger Federer')]
    # NadalGames = matches2012[(matches2012['Player 1'] == 'Rafael Nadal') | (matches2012['Player 2'] == 'Rafael Nadal')]
    # DjokovicGames = matches2012[(matches2012['Player 1'] == 'Novak Djokovic') | (matches2012['Player 2'] == 'Novak Djokovic')]

    # Points = pd.read_csv('ProjData/Points.csv')

    # FedererAux = FedererGames[['match_id','Player 1', 'Player 2','Surface']]
    # NadalAux = NadalGames[['match_id','Player 1', 'Player 2','Surface']]
    # DjokovicAux = DjokovicGames[['match_id','Player 1', 'Player 2','Surface']]

    # PointsFederer = pd.merge(Points,FedererAux, on='match_id')
    # PointsNadal = pd.merge(Points,NadalAux, on='match_id')
    # PointsDjokovic = pd.merge(Points,DjokovicAux, on='match_id')

    # Victor(PointsFederer,FedererGames)
    # Victor(PointsDjokovic,DjokovicGames)
    # Victor(PointsNadal,NadalGames)

    # FedererGames.to_csv('ProjData/Federer2012.csv',index=False)
    # NadalGames.to_csv('ProjData/Nadal2012.csv',index=False)
    # DjokovicGames.to_csv('ProjData/Djokovic2012.csv',index=False)

    # PointsFederer['Victor'] = PtWinner(PointsFederer,'Roger Federer', FedererGames)
    # PointsNadal['Victor'] = PtWinner(PointsNadal,'Rafael Nadal', NadalGames)
    # PointsDjokovic['Victor'] = PtWinner(PointsDjokovic,'Novak Djokovic', DjokovicGames)

    # PointsFederer['Server'] = Server(PointsFederer,'Roger Federer', FedererGames)
    # PointsNadal['Server'] = Server(PointsNadal,'Rafael Nadal', NadalGames)
    # PointsDjokovic['Server'] = Server(PointsDjokovic,'Novak Djokovic', DjokovicGames)

    # PointsFederer[['Ace','Winner','Unforced Error','Forced Error','Double Fault']] = None
    # PointsDjokovic[['Ace','Winner','Unforced Error','Forced Error','Double Fault']] = None
    # PointsNadal[['Ace','Winner','Unforced Error','Forced Error','Double Fault']] = None

    # PointsFederer.loc[(PointsFederer['2nd'].isna()), ['2nd']] = None
    # PointsNadal.loc[(PointsNadal['2nd'].isna()), ['2nd']] = None
    # PointsDjokovic.loc[(PointsDjokovic['2nd'].isna()), ['2nd']] = None

    # PtEnding(PointsFederer,EndDict)
    # PtEnding(PointsDjokovic,EndDict)
    # PtEnding(PointsNadal,EndDict)

    # PointsFederer.to_csv('ProjData/PointsFederer.csv',index=False)
    # PointsDjokovic.to_csv('ProjData/PointsDjokovic.csv',index=False)
    # PointsNadal.to_csv('ProjData/PointsNadal.csv',index=False)

    # PointsFederer.to_csv('ProjData/PointsFederer.csv',index=False)
    # PointsDjokovic.to_csv('ProjData/PointsDjokovic.csv',index=False)
    # PointsNadal.to_csv('ProjData/PointsNadal.csv',index=False)

In [131]:
Federer = 'Roger Federer'
Nadal = 'Rafael Nadal'
Djokovic = 'Novak Djokovic'

In [132]:
FedererGames = pd.read_csv('ProjData/Federer2012.csv')
NadalGames = pd.read_csv('ProjData/Nadal2012.csv')
DjokovicGames = pd.read_csv('ProjData/Djokovic2012.csv')

PointsFederer = pd.read_csv("ProjData/PointsFederer.csv")
PointsNadal = pd.read_csv("ProjData/PointsNadal.csv")
PointsDjokovic = pd.read_csv("ProjData/PointsDjokovic.csv")

In [133]:
Rally = pd.read_csv('BaseData/charting-m-stats-Rally.csv')
ShotTypes = pd.read_csv('BaseData\charting-m-stats-ShotTypes.csv')
Serves = pd.read_csv('BaseData/charting-m-stats-ServeBasics.csv')
ShotDir = pd.read_csv('BaseData/charting-m-stats-ShotDirection.csv')

In [134]:
FedererGamesRally = Rally[Rally['match_id'].isin(FedererGames['match_id'])]
FedererGamesServes = Serves[Serves['match_id'].isin(FedererGames['match_id'])]
FedererGamesShotTypes = ShotTypes[ShotTypes['match_id'].isin(FedererGames['match_id'])]
FedererGamesShotDir = ShotDir[ShotDir['match_id'].isin(FedererGames['match_id'])]


In [135]:
rowsServe = ['Total','1-3','4-6','7-9','10']
rowsIndx = ['match_id','1-3','4-6','7-9','10-']

In [136]:
    FedererRallyServe = FedererGamesRally[FedererGamesRally['server'] == Federer].drop(columns=['server','returner'])
    FedererRallyServe = FedererRallyServe[~FedererRallyServe['row'].isin(rowsServe)]
    FedererRallyServe['row'] = FedererRallyServe['row'].str[:3]
    FedererRallyServe = Rallys(FedererRallyServe,rowsIndx)

    FedererRallyReturn = FedererGamesRally[FedererGamesRally['returner'] == Federer].drop(columns=['server','returner'])
    FedererRallyReturn = FedererRallyReturn[~FedererRallyReturn['row'].isin(rowsServe)]
    FedererRallyReturn['row'] = FedererRallyReturn['row'].str[:3]
    FedererRallyReturn = Rallys(FedererRallyReturn,rowsIndx)

In [137]:
FedererShotDirTotal = FedererGamesShotDir[FedererGamesShotDir['player'] == Federer]
FedererShotDirTotal = FedererShotDirTotal[FedererShotDirTotal['row'] == 'Total']
FedererShotDirTotal = FedererShotDirTotal.drop(columns=['player','row'])
FedererShotDirTotal

,match_id,crosscourt,down_middle,down_the_line,inside_out,inside_in
32175,20121112-M-Tour_Finals-F-Roger_Federer-Novak_D...,137,96,36,51,4
32183,20121111-M-Tour_Finals-SF-Roger_Federer-Andy_M...,64,31,24,28,8
32223,20121106-M-Tour_Finals-RR-Roger_Federer-Janko_...,53,51,15,21,5
32287,20121028-M-Basel-F-Roger_Federer-Juan_Martin_D...,96,70,23,41,9
32319,20121012-M-Shanghai_Masters-SF-Roger_Federer-A...,58,34,12,21,4
32339,20121011-M-Shanghai_Masters-QF-Marin_Cilic-Rog...,52,31,15,14,0
32351,20121008-M-Shanghai_Masters-R16-Roger_Federer-...,61,46,26,31,5
32419,20120905-M-US_Open-QF-Tomas_Berdych-Roger_Federer,96,46,27,52,13
32439,20120830-M-US_Open-R64-Roger_Federer-Bjorn_Phau,65,24,8,40,11
32483,20120818-M-Cincinnati_Masters-F-Novak_Djokovic...,68,27,16,25,1


In [138]:
# text = '6n+f1f2b1b2-@'
# chars_to_remove = '+-'
# translation_table = str.maketrans('', '', chars_to_remove)
# updated_text = text.translate(translation_table)
# print(updated_text)

In [139]:
rowsST = ['match_id','Total', 'F', 'B', 'R', 'S', 'V','Z']
FedererGamesShotTypes = FedererGamesShotTypes[FedererGamesShotTypes['row'].isin(rowsST)].reset_index(drop=True)
FedererShotTypes = FedererGamesShotTypes[FedererGamesShotTypes['player'] == 'Roger Federer']
FedererShotStats = Shots(FedererShotTypes,rows=rowsST)
FedererShotStats = FedererShotStats.rename(columns=SymbolsDictionary)
FedererShotStats = FedererShotStats.fillna(0)|
FedererGames['Winner'] = FedererGames['Winner'].apply(lambda x: True if x == "['Roger Federer']" else False)
FedererShotStats['Win'] = FedererGames['Winner']
data = FedererShotStats[[ 'Total', 'Forehand', 'Backhand', 'FH Slice', 'BH Slice',
        'FH Volley', 'BH Volley', 'Win']]

SyntaxError: invalid syntax (4232636119.py, line 6)

In [ ]:

# F = forehand groundstroke (excluding slices, chips, etc.)
# b = backhand groundstroke (excluding slices, chips, etc.)

# r = forehand slice (including defensive chips, but not drop shots)
# s = backhand slice (including defensive chips, but not drop shots)
# v = forehand volley
# z = backhand volley
# (see below for an optional additional code to indicate a stop volley/drop volley)

# o = standard overhead/smash
# p = "backhand" overhead/smash
# u = forehand drop shot
# y = backhand drop shot
# l = forehand lob
# m = backhand lob
# h = forehand half-volley
# i = backhand half-volley
# j = forehand swinging volley
# k = backhand swinging volley
# t = all trick shots, including behind-the-back, between-the-legs, and "tweeners."


In [ ]:
FedererServes = ServStats(Serves,'Roger Federer',FedererGames)

In [ ]:
FedererServes

,match_id,Total,1st,Won1,2nd,Won2
0,20121112-M-Tour_Finals-F-Roger_Federer-Novak_D...,78,47,37,31,20
1,20121111-M-Tour_Finals-SF-Roger_Federer-Andy_M...,114,76,56,38,25
2,20121106-M-Tour_Finals-RR-Roger_Federer-Janko_...,95,63,53,32,18
3,20121028-M-Basel-F-Roger_Federer-Juan_Martin_D...,93,61,41,32,10
4,20121012-M-Shanghai_Masters-SF-Roger_Federer-A...,112,68,50,44,27
5,20121011-M-Shanghai_Masters-QF-Marin_Cilic-Rog...,157,100,68,57,33
6,20121008-M-Shanghai_Masters-R16-Roger_Federer-...,117,80,60,37,21
7,20120905-M-US_Open-QF-Tomas_Berdych-Roger_Federer,65,46,37,19,15
8,20120830-M-US_Open-R64-Roger_Federer-Bjorn_Phau,105,71,56,34,18
9,20120818-M-Cincinnati_Masters-F-Novak_Djokovic...,104,68,45,36,15


In [ ]:
df = pd.read_csv('Federer_data\FedererServe.csv')

In [ ]:
PlayerServes = ServStats(Serves,'Roger Federer',FedererGames)

In [ ]:
Serves

,match_id,player,row,pts,pts_won,aces,unret,forced_err,pts_won_lte_3_shots,wide,body,t
0,20241017-M-Campinas_CH-R16-Camilo_Ugo_Carabell...,Camilo Ugo Carabelli,Total,77,45,5,0,10,19,27,32,18
1,20241017-M-Campinas_CH-R16-Camilo_Ugo_Carabell...,Camilo Ugo Carabelli,1,48,32,5,0,9,17,20,10,18
2,20241017-M-Campinas_CH-R16-Camilo_Ugo_Carabell...,Camilo Ugo Carabelli,2,29,13,0,0,1,2,7,22,0
3,20241017-M-Campinas_CH-R16-Camilo_Ugo_Carabell...,Karue Sell,Total,59,36,2,0,12,16,26,15,18
4,20241017-M-Campinas_CH-R16-Camilo_Ugo_Carabell...,Karue Sell,1,36,27,2,0,9,13,17,7,12
...,...,...,...,...,...,...,...,...,...,...,...,...
34855,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,Nicola Pietrangeli,1,56,26,1,0,7,12,33,8,15
34856,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,Nicola Pietrangeli,2,27,15,0,0,0,1,5,20,2
34857,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,Luis Ayala,Total,85,41,2,0,6,13,41,15,29
34858,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,Luis Ayala,1,57,24,2,0,6,10,32,3,22


In [ ]:
PlayerServes

,match_id,Total,1st,Won1,2nd,Won2
0,20121112-M-Tour_Finals-F-Roger_Federer-Novak_D...,78,47,37,31,20
1,20121111-M-Tour_Finals-SF-Roger_Federer-Andy_M...,114,76,56,38,25
2,20121106-M-Tour_Finals-RR-Roger_Federer-Janko_...,95,63,53,32,18
3,20121028-M-Basel-F-Roger_Federer-Juan_Martin_D...,93,61,41,32,10
4,20121012-M-Shanghai_Masters-SF-Roger_Federer-A...,112,68,50,44,27
5,20121011-M-Shanghai_Masters-QF-Marin_Cilic-Rog...,157,100,68,57,33
6,20121008-M-Shanghai_Masters-R16-Roger_Federer-...,117,80,60,37,21
7,20120905-M-US_Open-QF-Tomas_Berdych-Roger_Federer,65,46,37,19,15
8,20120830-M-US_Open-R64-Roger_Federer-Bjorn_Phau,105,71,56,34,18
9,20120818-M-Cincinnati_Masters-F-Novak_Djokovic...,104,68,45,36,15
